# Необходимый импорт

In [1]:
import torch
import spacy
import re
import string
import pandas as pd
from pathlib import Path
from fastai.text.all import *
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from sklearn.metrics import precision_score, recall_score, f1_score

In [5]:
torch.randn(1).requires_grad_()

tensor([-0.0583], requires_grad=True)

In [18]:
torch.rand(3,3)

tensor([[0.1320, 0.6545, 0.5325],
        [0.1655, 0.8553, 0.3148],
        [0.7108, 0.9380, 0.9344]])

In [19]:
torch.tensor([i for i in range(1,10)]).view(-1,3)[1:3,1:3]

tensor([[5, 6],
        [8, 9]])

## Открытие файла

In [ ]:
path_to_folder = Path("../../SomeData/dataPod")

In [3]:
tec_sup_data = pd.read_csv(path_to_folder/"data.csv", sep=";", header=None)

In [6]:
tec_sup_data.columns

Index(['Тема', 'Описание', 'Категория', 'Тех. поддержка'], dtype='object')

In [ ]:
tec_sup_data[1].iloc[5]

In [ ]:
tec_sup_data = pd.read_excel(path_to_folder/"data.xlsx")
tec_sup_data.head()

In [ ]:
tec_sup_data[0:3]

### Подготавливаем данные

In [12]:
data_set = tec_sup_data.drop_duplicates()

In [13]:
display(tec_sup_data.shape)
display(data_set.shape)

(99354, 4)

(92536, 4)

In [63]:
#spacy.require_gpu()
nlpru = spacy.load("ru_core_news_lg")
stopwords = spacy.lang.ru.stop_words.STOP_WORDS
punctuations = string.punctuation+"-"+"..."+"”"+"”"+"№"
IP_SITE_REGEX = r"(?:(?:https?|ftp|http):\/\/)?[\w\/\-?=%.]+\.[\w\/\-&?=%.]+"#r"(.*[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b[-a-zA-Z0-9()@:%_\/+.~#?&=]*)"
MAIL_REGEX = r"([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)"
SCIP_DATA_REGEX = r"^\d+|^[Tt][Ee][Ss][Tt]|^[Пп][Рр][Оо][Вв][Ее][Рр][Кк][Аа]"
NUM_REGEX = r"\d+(?!\w)"#r"\d+(?= |\n|$)"
PUNCT_REGEX = r"[!\"#$%&\'()*+,-./:;<=>?@\[\\\]^_`{|}~\-...№«»„“‟”⹂•]"
NAMED_PAR_REGEX = r"(?<! )([А-ЯЁA-Z]+[а-яёa-z\._]+){2,}(?! )"#r"(\w+[_.])+([А-ЯЁ][а-яё]+))"#r"(\w+[_.])|((?<! )([А-ЯЁ][а-яё]+)(?! ))"
#NAMED2_PAR_REGEX = r"(?<! )([А-ЯЁ][а-яё]+){2,}(?! )"


def cleanup_text_sp(docs):
    tokens = nlpru(str(docs))
    text = []
    for tok in tokens:
        if not (re.search(NUM_REGEX, str(tok))) and not (re.search(IP_SITE_REGEX, str(tok)))\
            and not (re.search(r" {2,}", str(tok))):
            text.append(tok.text)
        elif (re.search(NUM_REGEX, str(tok))):
            text.append("ЧИСЛО")
        elif (re.search(IP_SITE_REGEX, str(tok))):
            text.append("АДРСАЙТА")
        elif (re.search(r" {2,}", str(tok))):
            continue
    #print(text)
    tokens = [tok for tok in text if tok not in (stopwords and punctuations)]
    #tokens = [tok for tok in tokens if tok not in SYMBOLS]  and not (re.search(IP_SITE_REGEX, str(tok))
    return " ".join(tokens)

def cleanup_text(docs:str):
    text_list = docs.split()
    output_text = []
    for item in text_list:
        #if re.sub(r'^\s+|\s+$', "", item.lower()) in stopwords:
        #    continue
        if not (re.search(NUM_REGEX + "|" + PUNCT_REGEX + "|" + IP_SITE_REGEX + "|" + NAMED_PAR_REGEX, str(item)))\
            and not (re.search(r" {2,}", str(item))):
            output_text.append(item)
            continue
        elif (re.search(IP_SITE_REGEX, str(item))):
            #tmp = re.sub(IP_SITE_REGEX, " АДРСАЙТА", item)
            #tmp = re.sub(NUM_REGEX, " ЧИСЛО", tmp)
            #tmp = re.sub(PUNCT_REGEX, "", tmp)
            output_text.append("АДРСАЙТА")
        elif (re.search(NUM_REGEX, str(item))):
            #tmp = re.sub(NUM_REGEX, "ЧИСЛО", item)
            #tmp = re.sub(PUNCT_REGEX, "", tmp)
            output_text.append("ЧИСЛО")
        elif (re.search(NAMED_PAR_REGEX, str(item))):
            output_text.append("ИМПАР")
        elif (re.search(PUNCT_REGEX, str(item))):
            temp = re.sub(PUNCT_REGEX, "", item)
            if temp == "":
                continue
            output_text.append(temp)
    return " ".join(output_text)

In [90]:
re.sub(r'^\s+|\s+$', "", "в".lower()) in stopwords

True

In [ ]:
cleanup_text(tec_sup_data["Описание"].iloc[65532])

In [ ]:
tec_sup_data["Описание"].iloc[65532]

In [87]:
string.punctuation+"-"+"..."+"”"+"”"+"№"

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~-...””№'

In [6]:
re.sub(r"(.+[_.])+", "ИМПАР", "ИСТОК_МодификацияОбъектов", count=0)

'ИМПАРМодификацияОбъектов'

In [69]:
index_rem = []
data_set = tec_sup_data.replace(r'^\s+|\s+$','',regex=True).copy()
#data_set = data_set[data_set["Тех. поддержка"] == "ОИТ"]
data_set['Категория'] = data_set['Категория'].astype( str )
data_set["Описание"] = data_set["Описание"].astype( str )
data_set["Описание"] = data_set["Описание"].apply(lambda x: cleanup_text(x))
#data_set["Описание"] = data_set["Описание"].replace(r'^\s+|\s+$','',regex=True)
#data_set["Описание"] = data_set["Описание"].replace(MAIL_REGEX,'ПОЧТА',regex=True)
#data_set["Описание"] = data_set["Описание"].replace(IP_SITE_REGEX,'АДРСАЙТА',regex=True)
#data_set["Описание"] = data_set["Описание"].replace(NAMED_PAR_REGEX,"ИМПАР",regex=True)
#data_set["Описание"] = data_set["Описание"].apply(lambda x: str(x).translate(str.maketrans('', '', string.punctuation+"№")))
#data_set["Описание"] = data_set["Описание"].replace(NUM_REGEX,'ЧИСЛО',regex=True)
#data_set["Описание"] = data_set["Описание"].replace(PUNCT_REGEX,'',regex=True)
#data_set["Описание"] = data_set["Описание"].replace(r" {2,}",' ',regex=True)
for data in data_set["Описание"]:
    if str(data)== "ЧИСЛО" or re.search(SCIP_DATA_REGEX, str(data)):
        for x in data_set[data_set["Описание"] == data].index:
            index_rem.append(x)
data_set.drop(index_rem, inplace=True)
data_set.reset_index(drop=True, inplace=True)
index_rem = None
#data_set['Категория'] = data_set['Категория'].str.replace('\"', '') 
data_set['Категория'] = data_set['Категория'].str.lower()
data_set["Описание"] = data_set["Описание"].str.lower()

In [7]:
data_set = pd.read_csv("../output/data_set.csv", sep=';')

In [ ]:
data_set.iloc[0]

In [95]:
data_set.to_csv(path_to_folder/'output/data_set.csv', header=True, sep=';', index=False)

In [ ]:
cleanup_text(tec_sup_data["Описание"].iloc[65532])

In [ ]:
docRu = nlpru(tec_sup_data["Описание"].iloc[65532])
spacy.displacy.render(docRu, style="dep", jupyter= True)

In [ ]:
nlpru(tec_sup_data["Описание"].iloc[0])

In [ ]:
cap_space("ФизическиеЛицаЗарплатаКадры")

In [63]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [132]:
data_set['Категория'] = data_set['Категория'].astype("category")
data_set["Категория"].cat.codes

0        50
1        50
2        47
3        47
4        47
         ..
98808    23
98809     6
98810    48
98811    23
98812    23
Length: 98813, dtype: int8

In [7]:
data_set = data_set.drop(["Тема", "Тех. поддержка"], axis=1)
data_set.shape

(88715, 2)

In [ ]:
cnt_cat = data_set['Категория'].value_counts().to_frame().reset_index()
drop_name = []
print(cnt_cat.shape)
for index in cnt_cat.index:
    if cnt_cat['count'][index] <= 500:
        drop_name.append(cnt_cat['Категория'][index])
print(drop_name)
for item in drop_name:
    data_set = data_set[data_set["Категория"] != item]
print(data_set["Категория"].value_counts().shape)
#for item in drop_name:
#    data_set["Категория"][(data_set["Категория"] == item)] = "Не определено"

In [ ]:
data_set

In [74]:
len(data_set["Категория"].value_counts())

22

In [71]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data_set["Категория"])
data_set["LableEncoder"] = le.transform(data_set["Категория"])

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(data_set, test_size=0.2, random_state=1)

X_train, X_val = train_test_split(X_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

print('Training Data Shape:', X_train.shape)
print('Testing Data Shape:', X_test.shape)
print(X_train.shape[0]//32)

Training Data Shape: (52155, 3)
Testing Data Shape: (17386, 3)
1629


In [ ]:
X_train[X_train["LableEncoder"]==14]

## Строим словарь из данных

In [28]:
len(data_set["Описание"])

95402

## Создаём датасет

In [ ]:
data_set[["Описание", "LableEncoder"]]

In [12]:
class SupTextDataset(Dataset):
    def __init__(self, tablet:pd.DataFrame, max_lable):
        self.max_index_lable = max_lable
        self.lable = tablet["LableEncoder"]
        self.text = tablet["Описание"]
      
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, idx):
        example = self.text.iloc[idx]
        lable = self.lable.iloc[idx]
        return example, int(lable)#tuple(0 if i != lable else 1 for i in range(0, self.max_index_lable))

In [13]:
train_iter = iter(SupTextDataset(X_train[["Описание", "LableEncoder"]], len(data_set["Категория"].value_counts())))

In [14]:
next(train_iter)

('обновление импар', 8)

In [15]:
next(train_iter)

('не включается пк', 2)

## Определяем tokenizer

In [31]:
data_set_iter = SupTextDataset(data_set[["Описание", "LableEncoder"]], len(data_set["Категория"].value_counts()))
tokenizer = get_tokenizer(tokenizer="spacy", language="ru_core_news_lg")

def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(cleanup_text(str(text)))

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"], max_tokens=20000)
vocab.set_default_index(vocab["<unk>"])

text_pipeline = lambda x: vocab(tokenizer(str(x)))

In [66]:
tokenizer = get_tokenizer(tokenizer="spacy", language="ru_core_news_lg")
text_pipeline = lambda x: vocab(tokenizer(str(x)))

In [ ]:
vocab.get_stoi()

In [33]:
len(vocab)

20000

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for _text, _label in batch:
        label_list.append(_label)
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)


train_iter = SupTextDataset(X_train[["Описание", "LableEncoder"]], len(data_set["Категория"].value_counts()))
dataloader = DataLoader(
    train_iter, batch_size=32, shuffle=False, drop_last=True, collate_fn=collate_batch
)

## Создаём модель

In [2]:
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.drop = nn.Dropout()
        self.fc0 = nn.Linear(embed_dim, 256)
        self.activ = nn.ReLU()
        self.drop2 = nn.Dropout()
        self.fc1 = nn.Linear(256, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc0.weight.data.uniform_(-initrange, initrange)
        self.fc0.bias.data.zero_()
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = self.fc0(embedded)
        x = self.activ(x)
        x = self.drop2(x)
        return self.fc1(x)

In [184]:
class OurFirstLSTM(nn.Module):
    def __init__(self, hidden_size, embedding_dim, vocab_size, num_class):
        super(OurFirstLSTM, self).__init__()
    
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.LSTM(input_size=embedding_dim,  
                hidden_size=hidden_size, num_layers=1, batch_first=True)
        self.predictor = nn.Linear(hidden_size, num_class)

    def forward(self, seq, offset):
        embeds = self.embedding(seq)
        lstm_out, (hn, cn) = self.encoder(embeds)
        lstm_out = lstm_out[-1]
        preds = self.predictor(lstm_out)
        return preds

## Тренируем модель

In [35]:
torch.cuda.empty_cache()

In [138]:
len(data_set["Категория"].value_counts())

22

In [185]:
model = OurFirstLSTM(100, 300, len(vocab), len(data_set["Категория"].value_counts())).to(device)
print(model)

OurFirstLSTM(
  (embedding): Embedding(52487, 300)
  (encoder): LSTM(300, 100, batch_first=True)
  (predictor): Linear(in_features=100, out_features=22, bias=True)
)


In [36]:
num_class = len(data_set["Категория"].value_counts())
vocab_size = len(vocab)
emsize = 512
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)
print(model)
print(f"vocab size:{vocab_size} embed_dim:{emsize} num class:{num_class}")

TextClassificationModel(
  (embedding): EmbeddingBag(20000, 512, mode='mean')
  (drop): Dropout(p=0.5, inplace=False)
  (fc0): Linear(in_features=512, out_features=256, bias=True)
  (activ): ReLU()
  (drop2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=256, out_features=15, bias=True)
)
vocab size:20000 embed_dim:512 num class:15


In [12]:
import time

# Hyperparameters
EPOCHS = 20  # epoch
LR = 5  # learning rate
BATCH_SIZE = 32  # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()


def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [38]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset


total_accu = None
train_iter, test_iter = SupTextDataset(X_train[["Описание", "LableEncoder"]], len(data_set["Категория"].value_counts())),\
      SupTextDataset(X_test[["Описание", "LableEncoder"]], len(data_set["Категория"].value_counts()))
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(
    train_dataset, [num_train, len(train_dataset) - num_train]
)

train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

| epoch   1 |   500/ 1549 batches | accuracy    0.374
| epoch   1 |  1000/ 1549 batches | accuracy    0.507
| epoch   1 |  1500/ 1549 batches | accuracy    0.566
-----------------------------------------------------------
| end of epoch   1 | time:  6.84s | valid accuracy    0.649 
-----------------------------------------------------------
| epoch   2 |   500/ 1549 batches | accuracy    0.603
| epoch   2 |  1000/ 1549 batches | accuracy    0.624
| epoch   2 |  1500/ 1549 batches | accuracy    0.633
-----------------------------------------------------------
| end of epoch   2 | time:  6.84s | valid accuracy    0.705 
-----------------------------------------------------------
| epoch   3 |   500/ 1549 batches | accuracy    0.666
| epoch   3 |  1000/ 1549 batches | accuracy    0.676
| epoch   3 |  1500/ 1549 batches | accuracy    0.670
-----------------------------------------------------------
| end of epoch   3 | time:  6.68s | valid accuracy    0.711 
-------------------------------

In [39]:
c
with open('../output/dict/classes_dict.pkl', 'wb') as f:
    pickle.dump(dict(zip([i for i in range(0,len(le.classes_))], le.classes_)), f)

In [ ]:
dict(zip([i for i in range(0,len(le.classes_))], le.classes_))

In [ ]:
print("Checking the results of test dataset.")
accu_test = evaluate(test_dataloader)
print("test accuracy {:8.3f}".format(accu_test))

In [76]:
import torch.nn.functional as nnf
our_label = dict(zip([i for i in range(0,len(le.classes_))], le.classes_))


def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        print(text)
        output = model(text, torch.tensor([0]))
        print(torch.max(nnf.softmax(output, dim=1), 1))
        print(output.shape)
        print(output)
        print(output.argmax())
        print(output.argmax(1))
        print(output.argmax(1).item())
        print(output[0][output.argmax(1).item()])
        print(output.argmin(1).item())
        print(output[0][output.argmin(1).item()])
        return output.argmax(1).item()


ex_text_str = cleanup_text("Прошу настроить портал Истока на компьютере . Очень срочно!")

model = model.to("cpu")

#print("Это %s" % predict(ex_text_str, text_pipeline))
print("Это %s" % our_label[predict(ex_text_str, text_pipeline)])

tensor([  3,  19, 243, 628,   0, 150,   0, 190])
torch.return_types.max(
values=tensor([0.3553]),
indices=tensor([17]))
torch.Size([1, 22])
tensor([[ 3.2610,  0.2617,  0.7748, -4.5049,  2.2341,  7.8900, -1.5943, -4.5011,
         -6.4304,  0.1897,  9.4881,  7.5935,  0.5631,  9.8114,  3.1982, -6.1653,
          1.8838,  9.9004, -1.5470, -7.0061, -5.0645, -5.1897]])
tensor(17)
tensor([17])
17
tensor(9.9004)
19
tensor(-7.0061)
Это удаленная тех. поддержка


In [121]:
cleanup_text("Протянуть витую пару в новый компьютер")

'Протянуть витую пару новый компьютер'

In [42]:
model(train_dataset[55][0]).shape

torch.Size([28, 21])

In [87]:
train_dataset[0][0]

tensor([   9, 1868,  145,    6,  380])

In [ ]:
batch_size = 3
num_epochs = 1
train_dataset = SupTextDataset(X_train[["Описание", "Num"]], len(data_set["Категория"].value_counts()), vocab)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True, collate_fn=collate_batch)
i = 0
print(next(train_loader._get_iterator()))
for epoch in range(num_epochs):
    for inputs, lable in train_loader:
        if i > 1:
            continue
        print(inputs,lable)
        i += 1

In [144]:
len(X_train[:(X_train.shape[0]//32)*32])

56192

In [126]:
len(vocab)

97159

In [59]:
te1, te2 = train_dataset[1]
print(te1, " ", te2)

Почта не работает письма не отправляет   11


In [69]:
len(train_dataset)

56192

In [125]:
torch.save(model.state_dict(), "../output/modle.chek")
torch.save(vocab, '../output/vocab.pth')

In [10]:
import pickle
with open('../output/dict/classes_dict.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)
vocab = torch.load('../output/vocab.pth')
num_class = len(loaded_dict)
vocab_size = len(vocab)
emsize = 512
model = TextClassificationModel(vocab_size, emsize , 22)
model_state_dict = torch.load("../output/modle.chek")
model.load_state_dict(model_state_dict)

<All keys matched successfully>